In [8]:
import datetime
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
import os

ee.Initialize()

# Define init params - area, satellite, years

In [9]:
# Area over which to aggregate data - this is the whole state
area = (ee.FeatureCollection('ft:1fRY18cjsHzDgGiJiS2nnpUU3v9JPDc2HNaR7Xk8').filter(ee.Filter().eq('Name', 'California')))

In [10]:
# County level
# area = (ee.FeatureCollection('ft:1FEPBzXYqUbdWXG2wTFOyb-ts9_GbeI5NFraMq2yk').filter(ee.Filter().eq('COUNTY num', 95)))

In [11]:
# import RS products
trmm = ee.ImageCollection('TRMM/3B42')
prism = ee.ImageCollection("OREGONSTATE/PRISM/AN81m")

In [12]:
# Set time range
years = [x for x in range(2000, 2016)]

### Helpers 

In [13]:
def filter_date(product,year):
    startyear = ee.Date.fromYMD(year,1,1)
    endyear =ee.Date.fromYMD(year+1,12,31)
    prod = product.filterDate(startyear, endyear).sort('system:time_start', False).select("ppt")
    return prod
    
def aggregate_precip(product,year):

    # Filter
    filtered = filter_date(product, year)

    # calculate the monthly mean
    def calcMean(imageCollection,year):
        mylist = ee.List([])
        months = range(1,13)
        for m in months:
                w = imageCollection.filter(ee.Filter.calendarRange(year, year, 'year')).filter(ee.Filter.calendarRange(m, m, 'month')).sum();
                mylist = mylist.add(w.set('year', year).set('month', m).set('date', ee.Date.fromYMD(year,m,1)).set('system:time_start',ee.Date.fromYMD(year,m,1)))
        return ee.ImageCollection.fromImages(mylist)

    # run the calcMonthlyMean function
    monthly = ee.ImageCollection(calcMean(filtered, year))

    # select the region of interest, 25000 is the cellsize in meters
    monthly = monthly.getRegion(area,3200,"epsg:4326").getInfo()

    return monthly 

def df_from_ee_object(imcol):
    df = pd.DataFrame(imcol, columns = imcol[0])
    df = df[1:]
    return(df)

def array_from_df(df, variable, month):
    df = df[df.id == str(month)] 
    
    # get data from df as arrays
    lons = np.array(df.longitude)
    lats = np.array(df.latitude)
    data = np.array(df[variable]) # Set var here 
                                              
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)

    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)

    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]

    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32)

    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
    
    return arr

# Main

In [14]:
finals = []
for year in years:
    finals.append(aggregate_precip(prism,year))

In [15]:
dfs = []
for i in finals:
    dfs.append(df_from_ee_object(i))

In [16]:
months = [int(x) for x in dfs[0].id.unique()]

In [17]:
yearly = []
for i in dfs:
    result = []
    for m in months:
        monthly = []
        a = array_from_df(i,"ppt",m)
        result.append(a)
    yearly.append(result)

In [18]:
yearly_arrays = [np.array(x) for x in yearly]

In [19]:
timeseries = [np.array(x) for x in yearly_arrays]

In [20]:
all_data = [item for sublist in timeseries for item in sublist]

In [21]:
final = np.dstack(all_data)

In [22]:
final = np.ma.masked_where(final == 0, final)

# Make a movie of precip over time

In [51]:
# create a ScalarMappable, initialize a data structure, fake the array of the scalar mappable, set max to 99th percentile of data
sm = plt.cm.ScalarMappable(cmap="spectral", norm=plt.Normalize(vmin=0, vmax=1200))#vmax=np.percentile(final, 99)))
sm._A = []
plt.clf()

s = np.shape(final)
nFrames = 12*16 # 12 months * 16 years
A = np.zeros((final.shape[0], final.shape[1], nFrames))

# Make the dates to be annotated
dates = []

mons = range(1,13)
yrs = range(2000,2016)

for y in yrs:
    for m in mons:
        dates.append(datetime.date(y,m,1))  #year, month, day


# Set up plotting
fig = plt.figure()
ax = plt.axis('off')
cbar = plt.colorbar(sm)
t = plt.title("monthly PRISM precipitation sums (mm)")

# Animation function
def animate(i): 
    result = plt.imshow(final[:,:,i],cmap = plt.get_cmap("spectral"))
    text = plt.text(300,20,str(dates[i])[:-3], ha='center', va='center', backgroundcolor='white')
    
    return result

anim = animation.FuncAnimation(fig, animate, frames=nFrames)

In [52]:
from IPython.display import HTML
HTML(anim.to_html5_video())

![title](prism.gif)

Now you have the monthly sums in a 3d array

# Visualize the monthly sums

In [53]:
fig, axs = plt.subplots(4,4, figsize=(20,15))
axs = axs.ravel()

plt.suptitle('California Precip in January (mm, 2000 - Present)', size = 50)

for i in range(len(dfs)):
    
    a = axs[i].scatter(dfs[i]['longitude'],dfs[i]['latitude'],marker = "s",c = dfs[i]['precipitation'],s= 20, lw = 0)
    axs[i].set_title(2000+i)
    axs[i].axis('off')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(a, cax=cbar_ax)

plt.show()

KeyError: 'precipitation'

# Write to csv

In [ ]:
dfs_by_year = dict(zip(years,dfs))
for k,v in dfs_by_year.items():
    path = os.path.join(os.getcwd(), "trmm_csvs")
    v.to_csv(os.path.join(path,str(k)+"_.csv"))